<a href="https://colab.research.google.com/github/amrutask/Machine_Learning/blob/master/HomeWork2/CNN_FinalArc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print("X_train Size :",X_train.shape)
print("X_test Size : ",X_test.shape)
print(y_train.shape)

170500096/170498071 [==============================] - 22s 0us/step
X_train Size : (50000, 32, 32, 3)
X_test Size :  (10000, 32, 32, 3)
(50000, 1)


In [4]:
num_classes=10
epoch=100

print("Size of train, validation and test labels: ", y_train.shape, y_test.shape) 

Size of train, validation and test labels:  (50000, 1) (10000, 1)


In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [6]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
scores = []
input_dim=X_train.shape[1:]
for train_index, val_index in kfold.split(X_train, y_train):
  print("___________________________________________________________________________________________________________________")
  trainx=X_train[train_index]
  trainy=y_train[train_index]
  valx=X_train[val_index]
  valy=y_train[val_index]
  trainy = keras.utils.to_categorical(trainy, num_classes)
  valy= keras.utils.to_categorical(valy, num_classes)
  
  model = Sequential()
  model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_dim, activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
  model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
  model.add(Dropout(0.25))
  model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(800, activation='relu'))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))

  opt = optimizers.rmsprop(lr=0.0001, decay=1e-6)

  model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
  
  datagen = ImageDataGenerator(featurewise_center=False,samplewise_center=False,featurewise_std_normalization=False,samplewise_std_normalization=False, zca_whitening=False,
        zca_epsilon=1e-06, rotation_range=10,width_shift_range=0.1, height_shift_range=0.2, shear_range=0., zoom_range=0.,channel_shift_range=0., fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0)

  datagen.fit(trainx)
  
  model.fit_generator(datagen.flow(trainx, trainy), steps_per_epoch=1000, epochs=100, validation_data=(valx, valy))
  score = model.evaluate(valx, valy, verbose=0) 
  scores.append(score[1])


___________________________________________________________________________________________________________________
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
1000/1000 [==============================] - 32s 32ms/step - loss: 1.9005 - acc: 0.3009 - val_loss: 1.6189 - val_acc: 0.4201
Epoch 2/100
1000/1000 [==============================] - 28s 28ms/step - loss: 1.6507 - acc: 0.4001 - val_loss: 1.4592 - val_acc: 0.4686
Epoch 3/100
1000/1000 [==============================] - 28s 28ms/step - loss: 1.5280 - acc: 0.4447 - val_loss: 1.5930 - val_acc: 0.4486
Epoch 4/100
1000/1000 [==============================] - 27s 27ms/step - loss: 1.4388 - acc: 0.4803 - val_loss: 1.2840 - val_acc: 0.5383
Epoch 5/100
1000/1000 [==============================] - 27s 27ms/step - loss: 1.3783 - acc: 0.5022 -

In [10]:
print("Overall validation accuracy:", sum(scores)/5)

y_test = keras.utils.to_categorical(y_test, num_classes)

score = model.evaluate(X_test, y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])


Overall validation accuracy: 0.81616
Test score: 0.6420029890298843
Test accuracy: 0.792
